In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/usr0/home/naveenr/projects/food_rescue_feedback')

In [3]:
import numpy as np
import random 
import matplotlib.pyplot as plt
from feedback.fr_feedback import *
import feedback.secret as secret
from feedback.database import open_connection, run_query
import openai
import csv
import datetime
from feedback.fr_dynamics import get_db_data, get_train_test_data_rating, get_data_predict_match_difficulty
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix

/usr0/home/naveenr/miniconda3/envs/food/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Data Cleanup

In [4]:
db_name = secret.database_name 
username = secret.database_username 
password = secret.database_password 
ip_address = secret.ip_address
port = secret.database_port

connection_dict = open_connection(db_name,username,password,ip_address,port)
connection = connection_dict['connection']
cursor = connection_dict['cursor']

In [6]:
donation_id_to_latlon, recipient_location_to_latlon, rescues_by_user, all_rescue_data, user_id_to_latlon = get_db_data() 

query = ("SELECT user_id, AVG(volunteer_rating) AS average_volunteer_rating FROM rescues WHERE volunteer_rating IS NOT NULL GROUP BY user_id")
user_data = run_query(cursor,query)
user_id_to_volunteer_rating = {}
for i in user_data:
    user_id_to_volunteer_rating[i['user_id']] = float(i['average_volunteer_rating'])

query = ("SELECT recipient_location_id, AVG(volunteer_rating) AS average_volunteer_rating FROM rescues WHERE volunteer_rating IS NOT NULL GROUP BY recipient_location_id")
recipient_data = run_query(cursor,query)
recipient_id_to_volunteer_rating = {}
for i in recipient_data:
    recipient_id_to_volunteer_rating[i['recipient_location_id']] = float(i['average_volunteer_rating'])


all_rescue_data = [i for i in all_rescue_data if i['volunteer_rating'] != None]
train_X, train_Y, valid_X, valid_Y, test_X, test_Y = get_train_test_data_rating(rescues_by_user,donation_id_to_latlon, recipient_location_to_latlon, user_id_to_latlon,all_rescue_data,user_id_to_volunteer_rating,recipient_id_to_volunteer_rating)

State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 9
State 9
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 9
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 9
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 9
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8
State 8


In [66]:
rf_classifier = RandomForestClassifier()
rf_classifier.fit(train_X, train_Y)
predictions = rf_classifier.predict(test_X)

matrix = confusion_matrix(predictions, test_Y)
matrix.diagonal()/matrix.sum(axis=1)

array([0.53271028, 0.45454545, 0.49403341, 0.8594179 ])

In [67]:
accuracy = accuracy_score(test_Y,predictions)
accuracy

0.8010110294117647

In [71]:
joblib_file = "../../data/models/difficulty_predictor.pkl"
joblib.dump(rf_classifier, joblib_file)

['../../data/models/difficulty_predictor.pkl']

In [72]:
list(rescues_by_user.keys())[0]

37683

In [75]:
list(donation_id_to_latlon.keys())[0]

1

In [92]:
user_id = 37683
donor_id = 1
recipient_location_id = 1315
state = 8.0
published_at = datetime.datetime.now()
rescue = {
    'donation_id': donor_id, 
    'recipient_location_id': recipient_location_id, 
    'state': state,
    'published_at': published_at
}
rf_classifier.predict([get_data_predict_match_difficulty(rescue,user_id, donation_id_to_latlon, recipient_location_to_latlon, user_id_to_latlon, rescues_by_user,user_id_to_volunteer_rating,recipient_id_to_volunteer_rating)])